In [1]:
import os
from zipfile import ZipFile
import io
import csv
import pandas as pd
from google.cloud import bigquery
from google.oauth2 import service_account
import numpy as np
import glob
import sqlite3 as sql

In [2]:
#this project begins with a collection of csv files on data from Emmaus, 
#an affordable housing non-profit and homeless shelter, from 2019-2020

#remove ".csv" endings from files (for easier titles in SQL)
for obj in os.listdir("Mitchsfiles2019/"):
    os.rename("Mitchsfiles2019/" + obj, "Mitchsfiles2019/" + obj.split(".")[0])
for obj in os.listdir("Mitchsfiles2020/"):
    os.rename("Mitchsfiles2020/" + obj, "Mitchsfiles2020/" + obj.split(".")[0])

In [3]:
#next is importing files to SQL, and executing a series of merges to get all 
#of the important information ready for analysis. I made this section unnecessarily bulky
#because sqlite will not let you simply remove columns from tables, so I have tons of "Select"
#statements to get only the variables of interest.

#connect to database
conn=sql.connect("/Users/peterkirgis/Documents/DataAnalytics/A+A_Project/MitchsFY2019.db")

for f in os.listdir("Mitchsfiles2019/"):
    df=pd.read_csv("Mitchsfiles2019/" + f)        
    df.to_sql(name=f,con=conn,if_exists='append', index=False)
    
#create cursor    
crsr=conn.cursor()

#define statement
d_table = '''CREATE TABLE IF NOT EXISTS "disability by participant" as 
SELECT DISTINCT D.EnrollmentID, C.PersonalID, C.FirstName, C.MiddleName, C.LastName, C.SSN, C.DOB, C.AmIndAKNative, C.Asian, C.BlackAfAmerican, C.NativeHIOtherPacific, C.White, C.Ethnicity, C.Gender, D.DisabilityType, D.DisabilityResponse, D.DataCollectionStage
FROM Client as C
LEFT JOIN Disabilities as D
ON C.PersonalID = D.PersonalID;'''
#execute statement
crsr.execute(d_table) 

edc_table = '''CREATE TABLE IF NOT EXISTS enrollment_disability_client as 
SELECT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, E.LengthOfStay, E.LivingSituation, E.TimesHomelessPastThreeYears, E.MonthsHomelessPastThreeYears, E.EntryDate
FROM "disability by participant" as EC
LEFT JOIN Enrollment as E
ON EC.EnrollmentID = E.EnrollmentID;'''
crsr.execute(edc_table)

eedc_table= '''CREATE TABLE IF NOT EXISTS exit_enrollment_disability_client as
SELECT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, E.ExitDate, E.Destination, EC.EntryDate
FROM enrollment_disability_client as EC
LEFT JOIN Exit as E
ON EC.ENrollmentID = E.EnrollmentID;'''
crsr.execute(eedc_table)

eedch_table= '''CREATE TABLE IF NOT EXISTS exit_enrollment_disability_client_healthDV as 
SELECT DISTINCT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, EC.ExitDate, EC.Destination, EC.EntryDate,
H.DomesticViolenceVictim, H.WhenOccurred, H.CurrentlyFleeing
FROM exit_enrollment_disability_client as EC
LEFT JOIN HealthAndDV as H
ON EC.EnrollmentID = H.EnrollmentID;'''
crsr.execute(eedch_table) 

ds_table='''CREATE TABLE IF NOT EXISTS disability_score as
SELECT EnrollmentID, DataCollectionStage, count(distinct(DisabilityType)) as client_disability_score
FROM Disabilities
WHERE DisabilityResponse == 1 OR DisabilityResponse == 2 OR DisabilityResponse == 3
GROUP BY EnrollmentID, DataCollectionStage;'''
crsr.execute(ds_table) 

eedchb_table='''CREATE TABLE IF NOT EXISTS exit_enrollment_disability_client_healthDV_benefits as 
SELECT DISTINCT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, EC.ExitDate, EC.Destination, EC.EntryDate,
EC.DomesticViolenceVictim, EC.WhenOccurred, EC.CurrentlyFleeing,
B.IncomeFromAnySource, B.TotalMonthlyIncome, B.Earned, B.EarnedAmount, B.Unemployment, B.UnemploymentAmount, B.SSI, B.SSIAmount, B.SSIAmount, 
B.SSDIAmount, B.VADisabilityService, B.VADisabilityNonService,
B.PrivateDisability, B.PrivateDisabilityAmount, B.WorkersComp, B.WorkersCompAmount, B.TANF, B.TANFAmount, B.GA, B.GAAmount, 
B.SocSecRetirement, B.SocSecRetirementAmount, B.Pension, B.PensionAmount, B.ChildSupport, B.Alimony, 
B.OtherIncomeSource, B.BenefitsFromAnySource, B.SNAP, B.WIC, B.TANFChildCare, 
B.TANFTransportation, B.OtherTANF, B.OtherBenefitsSource, B.InsuranceFromAnySource, B.Medicaid, B.Medicare, B.SCHIP, 
B.VAMedicalServices, B.EmployerProvided, B.COBRA, B.PrivatePay, B.StateHealthIns, B.IndianHealthServices, B.OtherInsurance, 
B.OtherInsuranceIdentify
FROM exit_enrollment_disability_client_healthDV as EC
LEFT JOIN IncomeBenefits as B
ON EC.EnrollmentID = B.EnrollmentID AND EC.DataCollectionStage = B.DataCollectionStage;'''
crsr.execute(eedchb_table) 

csee_table='''CREATE TABLE IF NOT EXISTS clients_entry_exit as
SELECT DISTINCT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, EC.ExitDate, EC.Destination, EC.EntryDate,
EC.DomesticViolenceVictim, EC.WhenOccurred, EC.CurrentlyFleeing,
B.IncomeFromAnySource, B.TotalMonthlyIncome, B.Earned, B.EarnedAmount, B.Unemployment, B.UnemploymentAmount, B.SSI, B.SSIAmount, B.SSIAmount, 
B.SSDIAmount, B.VADisabilityService, B.VADisabilityNonService,
B.PrivateDisability, B.PrivateDisabilityAmount, B.WorkersComp, B.WorkersCompAmount, B.TANF, B.TANFAmount, B.GA, B.GAAmount, 
B.SocSecRetirement, B.SocSecRetirementAmount, B.Pension, B.PensionAmount, B.ChildSupport, B.Alimony, 
B.OtherIncomeSource, B.BenefitsFromAnySource, B.SNAP, B.WIC, B.TANFChildCare, 
B.TANFTransportation, B.OtherTANF, B.OtherBenefitsSource, B.InsuranceFromAnySource, B.Medicaid, B.Medicare, B.SCHIP, 
B.VAMedicalServices, B.EmployerProvided, B.COBRA, B.PrivatePay, B.StateHealthIns, B.IndianHealthServices, B.OtherInsurance, 
B.OtherInsuranceIdentify
FROM exit_enrollment_disability_client_healthDV as EC
LEFT JOIN IncomeBenefits as B
ON EC.EnrollmentID = B.EnrollmentID AND EC.DataCollectionStage = B.DataCollectionStage;'''
crsr.execute(csee_table) 

cee_table='''CREATE TABLE IF NOT EXISTS client_entry_exit as
SELECT DISTINCT B.EnrollmentID, B.PersonalID, B.FirstName, B.MiddleName, B.LastName, B.SSN, B.DOB, B.AmIndAKNative, B.Asian, B.BlackAfAmerican, B.NativeHIOtherPacific, B.White, B.Ethnicity, B.Gender, B.DataCollectionStage, B.LengthOfStay, B.LivingSituation, B.TimesHomelessPastThreeYears, B.MonthsHomelessPastThreeYears, B.ExitDate, B.Destination, B.EntryDate,
B.DomesticViolenceVictim, B.WhenOccurred, B.CurrentlyFleeing,
B.IncomeFromAnySource, B.TotalMonthlyIncome, B.Earned, B.EarnedAmount, B.Unemployment, B.UnemploymentAmount, B.SSI, B.SSIAmount, B.SSIAmount, 
B.SSDIAmount, B.VADisabilityService, B.VADisabilityNonService,
B.PrivateDisability, B.PrivateDisabilityAmount, B.WorkersComp, B.WorkersCompAmount, B.TANF, B.TANFAmount, B.GA, B.GAAmount, 
B.SocSecRetirement, B.SocSecRetirementAmount, B.Pension, B.PensionAmount, B.ChildSupport, B.Alimony, 
B.OtherIncomeSource, B.BenefitsFromAnySource, B.SNAP, B.WIC, B.TANFChildCare, 
B.TANFTransportation, B.OtherTANF, B.OtherBenefitsSource, B.InsuranceFromAnySource, B.Medicaid, B.Medicare, B.SCHIP, 
B.VAMedicalServices, B.EmployerProvided, B.COBRA, B.PrivatePay, B.StateHealthIns, B.IndianHealthServices, B.OtherInsurance, 
B.OtherInsuranceIdentify,  D.client_disability_score
FROM clients_entry_exit as B
LEFT JOIN disability_score as D
ON B.EnrollmentID = D.EnrollmentID AND B.DataCollectionStage = D.DataCollectionStage'''
crsr.execute(cee_table) 

#commit statements
conn.commit()

#close connection
conn.close()

In [4]:
#repeat process for 2020 data
conn=sql.connect("/Users/peterkirgis/Documents/DataAnalytics/A+A_Project/MitchsFY2020.db")

for f in os.listdir("Mitchsfiles2020/"):
    df=pd.read_csv("Mitchsfiles2020/" + f)        
    df.to_sql(name=f,con=conn,if_exists='append', index=False)
    
crsr=conn.cursor()

d_table = '''CREATE TABLE IF NOT EXISTS "disability by participant" as 
SELECT DISTINCT D.EnrollmentID, C.PersonalID, C.FirstName, C.MiddleName, C.LastName, C.SSN, C.DOB, C.AmIndAKNative, C.Asian, C.BlackAfAmerican, C.NativeHIOtherPacific, C.White, C.Ethnicity, C.Gender, D.DisabilityType, D.DisabilityResponse, D.DataCollectionStage
FROM Client as C
LEFT JOIN Disabilities as D
ON C.PersonalID = D.PersonalID;'''
crsr.execute(d_table) 

edc_table = '''CREATE TABLE IF NOT EXISTS enrollment_disability_client as 
SELECT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, E.LengthOfStay, E.LivingSituation, E.TimesHomelessPastThreeYears, E.MonthsHomelessPastThreeYears, E.EntryDate
FROM "disability by participant" as EC
LEFT JOIN Enrollment as E
ON EC.EnrollmentID = E.EnrollmentID;'''
crsr.execute(edc_table)

eedc_table= '''CREATE TABLE IF NOT EXISTS exit_enrollment_disability_client as
SELECT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, E.ExitDate, E.Destination, EC.EntryDate
FROM enrollment_disability_client as EC
LEFT JOIN Exit as E
ON EC.ENrollmentID = E.EnrollmentID;'''
crsr.execute(eedc_table)

eedch_table= '''CREATE TABLE IF NOT EXISTS exit_enrollment_disability_client_healthDV as 
SELECT DISTINCT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, EC.ExitDate, EC.Destination, EC.EntryDate,
H.DomesticViolenceVictim, H.WhenOccurred, H.CurrentlyFleeing
FROM exit_enrollment_disability_client as EC
LEFT JOIN HealthAndDV as H
ON EC.EnrollmentID = H.EnrollmentID;'''
crsr.execute(eedch_table) 

ds_table='''CREATE TABLE IF NOT EXISTS disability_score as
SELECT EnrollmentID, DataCollectionStage, count(distinct(DisabilityType)) as client_disability_score
FROM Disabilities
WHERE DisabilityResponse == 1 OR DisabilityResponse == 2 OR DisabilityResponse == 3
GROUP BY EnrollmentID, DataCollectionStage;'''
crsr.execute(ds_table) 

eedchb_table='''CREATE TABLE IF NOT EXISTS exit_enrollment_disability_client_healthDV_benefits as 
SELECT DISTINCT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DisabilityType, EC.DisabilityResponse, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, EC.ExitDate, EC.Destination, EC.EntryDate,
EC.DomesticViolenceVictim, EC.WhenOccurred, EC.CurrentlyFleeing,
B.IncomeFromAnySource, B.TotalMonthlyIncome, B.Earned, B.EarnedAmount, B.Unemployment, B.UnemploymentAmount, B.SSI, B.SSIAmount, B.SSIAmount, 
B.SSDIAmount, B.VADisabilityService, B.VADisabilityNonService,
B.PrivateDisability, B.PrivateDisabilityAmount, B.WorkersComp, B.WorkersCompAmount, B.TANF, B.TANFAmount, B.GA, B.GAAmount, 
B.SocSecRetirement, B.SocSecRetirementAmount, B.Pension, B.PensionAmount, B.ChildSupport, B.Alimony, 
B.OtherIncomeSource, B.BenefitsFromAnySource, B.SNAP, B.WIC, B.TANFChildCare, 
B.TANFTransportation, B.OtherTANF, B.OtherBenefitsSource, B.InsuranceFromAnySource, B.Medicaid, B.Medicare, B.SCHIP, 
B.VAMedicalServices, B.EmployerProvided, B.COBRA, B.PrivatePay, B.StateHealthIns, B.IndianHealthServices, B.OtherInsurance, 
B.OtherInsuranceIdentify
FROM exit_enrollment_disability_client_healthDV as EC
LEFT JOIN IncomeBenefits as B
ON EC.EnrollmentID = B.EnrollmentID AND EC.DataCollectionStage = B.DataCollectionStage;'''
crsr.execute(eedchb_table) 

csee_table='''CREATE TABLE IF NOT EXISTS clients_entry_exit as
SELECT DISTINCT EC.EnrollmentID, EC.PersonalID, EC.FirstName, EC.MiddleName, EC.LastName, EC.SSN, EC.DOB, EC.AmIndAKNative, EC.Asian, EC.BlackAfAmerican, EC.NativeHIOtherPacific, EC.White, EC.Ethnicity, EC.Gender, EC.DataCollectionStage, EC.LengthOfStay, EC.LivingSituation, EC.TimesHomelessPastThreeYears, EC.MonthsHomelessPastThreeYears, EC.ExitDate, EC.Destination, EC.EntryDate,
EC.DomesticViolenceVictim, EC.WhenOccurred, EC.CurrentlyFleeing,
B.IncomeFromAnySource, B.TotalMonthlyIncome, B.Earned, B.EarnedAmount, B.Unemployment, B.UnemploymentAmount, B.SSI, B.SSIAmount, B.SSIAmount, 
B.SSDIAmount, B.VADisabilityService, B.VADisabilityNonService,
B.PrivateDisability, B.PrivateDisabilityAmount, B.WorkersComp, B.WorkersCompAmount, B.TANF, B.TANFAmount, B.GA, B.GAAmount, 
B.SocSecRetirement, B.SocSecRetirementAmount, B.Pension, B.PensionAmount, B.ChildSupport, B.Alimony, 
B.OtherIncomeSource, B.BenefitsFromAnySource, B.SNAP, B.WIC, B.TANFChildCare, 
B.TANFTransportation, B.OtherTANF, B.OtherBenefitsSource, B.InsuranceFromAnySource, B.Medicaid, B.Medicare, B.SCHIP, 
B.VAMedicalServices, B.EmployerProvided, B.COBRA, B.PrivatePay, B.StateHealthIns, B.IndianHealthServices, B.OtherInsurance, 
B.OtherInsuranceIdentify
FROM exit_enrollment_disability_client_healthDV as EC
LEFT JOIN IncomeBenefits as B
ON EC.EnrollmentID = B.EnrollmentID AND EC.DataCollectionStage = B.DataCollectionStage;'''
crsr.execute(csee_table) 

cee_table='''CREATE TABLE IF NOT EXISTS client_entry_exit as
SELECT DISTINCT B.EnrollmentID, B.PersonalID, B.FirstName, B.MiddleName, B.LastName, B.SSN, B.DOB, B.AmIndAKNative, B.Asian, B.BlackAfAmerican, B.NativeHIOtherPacific, B.White, B.Ethnicity, B.Gender, B.DataCollectionStage, B.LengthOfStay, B.LivingSituation, B.TimesHomelessPastThreeYears, B.MonthsHomelessPastThreeYears, B.ExitDate, B.Destination, B.EntryDate,
B.DomesticViolenceVictim, B.WhenOccurred, B.CurrentlyFleeing,
B.IncomeFromAnySource, B.TotalMonthlyIncome, B.Earned, B.EarnedAmount, B.Unemployment, B.UnemploymentAmount, B.SSI, B.SSIAmount, B.SSIAmount, 
B.SSDIAmount, B.VADisabilityService, B.VADisabilityNonService,
B.PrivateDisability, B.PrivateDisabilityAmount, B.WorkersComp, B.WorkersCompAmount, B.TANF, B.TANFAmount, B.GA, B.GAAmount, 
B.SocSecRetirement, B.SocSecRetirementAmount, B.Pension, B.PensionAmount, B.ChildSupport, B.Alimony, 
B.OtherIncomeSource, B.BenefitsFromAnySource, B.SNAP, B.WIC, B.TANFChildCare, 
B.TANFTransportation, B.OtherTANF, B.OtherBenefitsSource, B.InsuranceFromAnySource, B.Medicaid, B.Medicare, B.SCHIP, 
B.VAMedicalServices, B.EmployerProvided, B.COBRA, B.PrivatePay, B.StateHealthIns, B.IndianHealthServices, B.OtherInsurance, 
B.OtherInsuranceIdentify,  D.client_disability_score
FROM clients_entry_exit as B
LEFT JOIN disability_score as D
ON B.EnrollmentID = D.EnrollmentID AND B.DataCollectionStage = D.DataCollectionStage'''
crsr.execute(cee_table) 
conn.commit() 
conn.close()